In [ ]:
#————————————————————

# Name: Azure OpenAI API, Fine-Tuning

# Purpose: This notebook will use the Fine-tuned to create vegan recipes.

# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024 & ZHAW 2025
# Date Created: 22.01.2024
# Last Updated: 19.01.2025
# Python Version: 3.12.1

# Additionals:
# Use Region Sweden Central

# If necessary, download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\sdsc\requirements.txt

#————————————————————

In [1]:
# Import Python packages
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI
import json
from IPython.display import clear_output
from IPython.core.display import HTML
import requests
import random

In [2]:
# Load required variables from .env file.
load_dotenv(dotenv_path=Path("/workspaces/azure-openai-lab/.venv/.env")) #Error sometimes due to \ or \\. Try one or the other. "C:\\Python\\azure-openai-lab\\.venv\\.env"

# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
azure_oai_key = os.environ['AZURE_OPENAI_KEY_P34']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT_P34']

In [3]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key = azure_oai_key,  
    azure_endpoint = azure_oai_endpoint,
    api_version ="2024-05-01-preview" #"2024-02-15-preview"
    )

In [4]:
# Zero-Shot learning. Model has a token limit of 4096.

# Create advanced System prompt
systemcontent = \
"""
### INSTRUCTIONS
Persona: Act as a head chef such as Joël Robuchon who specializes in simple contemporary cuisine.
Action: Create well-thought-out and flavourful vegan recipes from a list of ingredients implementing classic culinary techniques.
Target Audience: The recipients of these vegan recipes are couples who want to cook a special meal at least once a week.

---

### OUTPUT FORMAT
Output only one vegan recipe and return it as a JSON object with the following format:
{"name":"","minutes":,"tags":"[]","nutrition":"[]","n_steps":"","steps":"[]","description":"","ingredients":"[]", "n_ingredients":}

The variables should contain the following information:
- name: the name of the recipe.
- minutes: the time in minutes to prepare the recipe.
- tags: a list of words that characterize the recipe.
- nutrition: a list of numeric values representing calories, total fat, sugar, sodium, protein, saturated fat, and carbohydrates.
- n_steps: the number of steps to prepare the recipe.
- steps: a list of steps to prepare the recipe.
- description: a summary of the recipe.
- ingredients: a list of the ingredient names in the recipe.
- n_ingredients: the total number of ingredients used in the recipe.
"""

# Create a prompt of ingredients the model should create a recipe from
ingredients = """'Tofu', 'Avocado', 'Soy Sauce', 'Chili', 'Coconut Milk', 'Broccoli'"""

# Send request to Azure OpenAI model
completion = client.chat.completions.create(
  model = "gpt-4o-mini-ft",
  #response_format={ "type": "json_object" }, # Not support for fine tuned models
  messages = [    
    {"role": "system", "content": systemcontent},
    {"role": "user", "content": ingredients}
  ]
)

# View generated recipe
result = completion.choices[0].message.content
print(result)


{"name":"avocado tofu & broccoli stir fry","minutes":15,"tags":"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'print-friendly', 'main-dish', 'beans', 'easy', 'healthy', 'entree', 'vegan', 'vegetarian', 'dietary-restrictions', 'soy-tofu', 'stir-fry', 'low-calorie', 'low-in-something', 'low-fat', 'low-cholesterol', 'low-saturated-fat', 'healthy-2', 'low-sodium', 'comfort-food', 'taste-mood', 'preparation', 'for-large-groups', '3-steps-or-less', 'after-school', '4-hours-or-less']","nutrition":"[153.9, 11.0, 2.0, 492.0, 30.0, 1.0, 13.0]","n_steps":7,"steps":"['heat the oil in a large skillet or wok over high heat', 'add the broccoli and the chili and fry for 1-2 minutes', 'add the tofu and fry for a further 5-6 minutes or until the tofu and broccoli are slightly browned', 'reduce the heat to medium-high and pour in the coconut milk and soy sauce stir well and simmer for 1-2 minutes', 'add the avocado and cook for a further minute', \"add water here if you'd like to ma

In [5]:
#Transform output to pandas dataframe and save as CSV file

# Clean up Azure OpenAI Output
json_data = result.strip('` \n')

if json_data.startswith('json'):
    json_data = json_data[4:]  # Remove the first 4 characters 'json'

recipes_from_ft_json = json.loads(json_data)
recipes_from_ft = pd.json_normalize(recipes_from_ft_json)
# path_output = r"/workspaces/azure-openai-lab/data/recipes-from-ft.csv" #r"C:\Python\azure-openai-lab\data\recipes-from-ft.csv"
# recipes_from_ft.to_csv(path_output, sep='\t', encoding='utf-8', index=False)